In [8]:
import datetime
import numpy as np
import pandas as pd
import requests
import scipy
import scipy.sparse as sp
import json

import os
import pickle
import json


from bs4 import BeautifulSoup

# data processing
from utils import weatherapi

from sklearn import preprocessing


ImportError: Could not import c:\Users\dusty3ntity\Desktop\projects\Air_alarms\isw_3_data_preprocessing.ipynb for ipynb.fs.full.isw_3_data_preprocessing: not a valid ipynb file

In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [31]:
OUTPUT_FOLDER = "data/prediction"
if not os.path.exists(OUTPUT_FOLDER):
   os.makedirs(OUTPUT_FOLDER)

ISW_DATA_FILE = "data/isw/reports_processed_v2.csv"
WEATHER_EVENTS_DATA_FILE = "data/alarms/merged_events.csv"


tfidf_transformer_model = "tfidf_transformer"
tfidf_transformer_version = "v1"

alarm_classifier = "alarm_classifier"
alarm_classifier_version = "v1"

count_vectorizer_model = "count_vectorizer"
count_vectorizer_version = "v1"

MODELS_FOLDER = "data/models"

decision_tree_model = "DecisionTreeClassifier"
decision_tree_version = "v1"

## 1. Load models & data

In [32]:
clf = pickle.load(open(f"data/{alarm_classifier}_{alarm_classifier_version}.pkl", "rb"))
tfidf = pickle.load(open(f"data/isw/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"data/isw/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

label_encoder = pickle.load(open(f"data/isw/hour_conditions_label_encoder.pkl", "rb"))

decision_tree = pickle.load(open(f"{MODELS_FOLDER}/{decision_tree_model}.pkl", "rb"))


In [5]:
REGIONS_DICTIONARY_FILE = "data/meta/regions.csv"
df_regions = pd.read_csv(REGIONS_DICTIONARY_FILE)
df_regions.head(25)

,region,center_city_ua,center_city_en,region_alt,region_id
0,АР Крим,Сімферополь,Simferopol,Крим,1
1,Вінницька,Вінниця,Vinnytsia,Вінниччина,2
2,Волинська,Луцьк,Lutsk,Волинь,3
3,Дніпропетровська,Дніпро,Dnipro,Дніпропетровщина,4
4,Донецька,Донецьк,Donetsk,Донеччина,5
5,Житомирська,Житомир,Zhytomyr,Житомирщина,6
6,Закарпатська,Ужгород,Uzhgorod,Закарпаття,7
7,Запорізька,Запоріжжя,Zaporozhye,Запоріжжя,8
8,Івано-Франківська,Івано-Франківськ,Ivano-Frankivsk,Овано-Франківщина,9
9,Київська,Київ,Kyiv,Київщина,10


## 2. Setup prediction region

In [24]:
region = "Львівщина"
date = "2023-04-09"

city = df_regions[df_regions["region_alt"]==region]["center_city_en"].values[0]

## 3. Get weather

In [35]:
level = "hours"
location = f"{city},Ukraine"

file_name = f"weather___{city.lower()}__{date}.json"

In [38]:
if not os.path.isfile(f"{OUTPUT_FOLDER}/{file_name}"):
	city_weather_json = weatherapi.get_weather_json(location, date)
	json_object = json.dumps(city_weather_json, indent=4)

	with open(f"{OUTPUT_FOLDER}/{file_name}", "w") as outfile:
		outfile.write(json_object)
else:
	print(f"Weather data from the \nregion {region}; \ndate {date}; \n is ready")

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Lviv,Ukraine/1681026667/1681069867?key=3QULGS8KJPBECC4J3W7MPEF2T


In [ ]:
weather_file_path = f"{OUTPUT_FOLDER}/{file_name}"
weather_for_day_hourly = weather.get_weather_hourly_for_region(weather_file_path)
weather_df = pd.DataFrame.from_dict(weather_for_day_hourly)

weather_df["day_datetime"] = pd.to_datetime(weather_df["day_datetime"])
weather_df["city"] = weather_df["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
weather_df["city"] = weather_df["city"].replace("Хмельницька область", "Хмельницький")

weather_df_v2 = weather.exclude_fields(weather_df)

In [ ]:
weather_df_v3 = pd.merge(weather_df_v2, df_regions, left_on="city", right_on="center_city_ua")

## 3. Get report

In [44]:
date = "2023-04-08"

BASE_URL = "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment"
months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
date_arr = date.split("-")

url = f"{BASE_URL}-{months[int(date_arr[1])-1]}-{int(date_arr[2])}-{int(date_arr[0])}"
print(url)

page = requests.get(url)
isw_report_html_file = f"{OUTPUT_FOLDER}/isw___{date}.html"
with open(isw_report_html_file, "wb+") as f:
  f.write(page.content)

https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-april-8-2023


In [ ]:
with open(isw_report_html_file, "r") as cfile:
	isw_report_html_txt = cfile.read()
	parsed_html = BeautifulSoup(isw_report_html_txt)

	title, link, text_title, text_main = tp.parse_html(parsed_html)
	main_html_v2 = tp.remove_names_and_date(str(text_main))

In [ ]:
main_html_v2 = BeautifulSoup(str(main_html_v2)).text

main_html_v3 = tp.replace_date(main_html_v2, date)

main_html_v3 = tp.preprocess_first(main_html_v3)
main_html_v3_lemm = tp.preprocess_second(main_html_v3, "lemm")

repowr_tf_idf = tf_idf.conver_doc_to_vector(main_html_v3_lemm, cv, tfidf)

In [ ]:
repowr_tf_idf

In [ ]:
word_count_vector = cv.transform([main_html_v3_lemm])
tfidf_vector = tfidf.transform(word_count_vector)

## 4. Data preparation

In [45]:
fields_to_exclude = [
	"city_resolvedAddress",
	"day_datetime",
	"day_datetimeEpoch",
	"hour_datetime",
	"hour_datetimeEpoch",
	"city",
	"region",
	"center_city_ua",
	"center_city_en"
]

tmp_fields_to_exclude = [
	"day_sunrise",
	"day_sunset",
	"hour_preciptype",
	"hour_conditions",
	"hour_solarenergy",
	"region_alt"
]

df_work_v2 = weather_df_v3.drop(fields_to_exclude, axis=1)

In [46]:
df_work_v2["hour_conditions"] = df_work_v2["hour_conditions"].apply(lambda x: x.split(",")[0])
df_work_v2["hour_conditions_id"] = label_encoder.transform(df_work_v2["hour_conditions"])

NameError: name 'df_work_v2' is not defined

In [47]:
df_work_v3 - df_work_v2.drop(tmp_fields_to_exclude, axis=1)

NameError: name 'df_work_v3' is not defined

In [ ]:
df_work_v3.head(10)

In [ ]:
tfidf_matrix = tfidf_vector

for i in range(0, 23):
	tfidf_matrix = sp.vstack((tfidf_matrix, tfidf_vector), format="csr")

In [ ]:
df_work_v4_csr = scipy.sparce.csr_matrix(df_work_v3.values)
df_all_features = sp.hstack((df_work_v4_csr, tfidf_matrix), format="csr")

In [ ]:
hours_alarm_schedule = clf.predict(df_all_features)

In [ ]:
hours_alarm_schedule